In [2]:
import pandas as pd
data_path = "0_data/"

developing_ft = data_path + "developing_2017stall.csv"
developing_ft_df = pd.read_csv(developing_ft)

developing_pred = data_path + "developing_2017pred.csv"
developing_pred_df = pd.read_csv(developing_pred)

In [3]:
from osgeo import gdal
import numpy as np
block_raster = gdal.Open(data_path + 'block_raster.tif')
block_raster_band = block_raster.GetRasterBand(1)
block_raster_data = block_raster.ReadAsArray()
row_length = block_raster.RasterYSize
col_length = block_raster.RasterXSize
tif_geotrans = block_raster.GetGeoTransform()
tif_proj = block_raster.GetProjection()
lat_max = 629 # For calculating row
lon_min = 9 # For calculating col

In [4]:
# 1. All results.
all_pred = np.zeros((row_length, col_length))
for i in range(len(developing_pred_df)):
    pred_obs = float(developing_pred_df.iat[i, 1])
    row = lat_max - developing_ft_df.iloc[i].at['latit']
    col = developing_ft_df.iloc[i].at['longit'] - lon_min
    all_pred[row, col] = all_pred[row, col] + pred_obs


# Save to tif
driver = gdal.GetDriverByName('GTiff')
developing_2017pred_dataset = driver.Create(data_path + 'developing_2017pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
developing_2017pred_dataset.SetGeoTransform(tif_geotrans)
developing_2017pred_dataset.SetProjection(tif_proj)
developing_2017pred_dataset.GetRasterBand(1).WriteArray(all_pred)
# Save finish.
del developing_2017pred_dataset

In [5]:
# 2. Workingday and weekend results.
workday_pred_sum = np.zeros((row_length, col_length))
workday_pred_num = np.zeros((row_length, col_length))
weekend_pred_sum = np.zeros((row_length, col_length))
weekend_pred_num = np.zeros((row_length, col_length))
for i in range(len(developing_pred_df)):
    holiday = int(developing_ft_df.iloc[i].at['holiday'])
    dow = int(developing_ft_df.iloc[i].at['dow'])
    if holiday == 0:
        if dow != 0 and dow != 6:
            pred_obs = float(developing_pred_df.iat[i, 1])
            row = lat_max - developing_ft_df.iloc[i].at['latit']
            col = developing_ft_df.iloc[i].at['longit'] - lon_min
            workday_pred_sum[row, col] = workday_pred_sum[row, col] + pred_obs
            workday_pred_num[row, col] = workday_pred_num[row, col] + 1
        else:
            pred_obs = float(developing_pred_df.iat[i, 1])
            row = lat_max - developing_ft_df.iloc[i].at['latit']
            col = developing_ft_df.iloc[i].at['longit'] - lon_min
            weekend_pred_sum[row, col] = workday_pred_sum[row, col] + pred_obs
            weekend_pred_num[row, col] = weekend_pred_num[row, col] + 1

workday_pred_avg = np.zeros((row_length, col_length))
weekend_pred_avg = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        if workday_pred_num[row, col] > 0:
            workday_pred_avg[row, col] = workday_pred_sum[row, col]/float(workday_pred_num[row, col])
        if weekend_pred_num[row, col] > 0:
            weekend_pred_avg[row, col] = weekend_pred_sum[row, col]/float(weekend_pred_num[row, col])

# Save to tif
driver = gdal.GetDriverByName('GTiff')
workday_pred_dataset = driver.Create(data_path + 'developing_workday_avg_pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
workday_pred_dataset.SetGeoTransform(tif_geotrans)
workday_pred_dataset.SetProjection(tif_proj)
workday_pred_dataset.GetRasterBand(1).WriteArray(workday_pred_avg)
# Save finish.
del workday_pred_dataset

# Save to tif
driver = gdal.GetDriverByName('GTiff')
weekend_pred_dataset = driver.Create(data_path + 'developing_weekend_avg_pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
weekend_pred_dataset.SetGeoTransform(tif_geotrans)
weekend_pred_dataset.SetProjection(tif_proj)
weekend_pred_dataset.GetRasterBand(1).WriteArray(weekend_pred_avg)
# Save finish.
del weekend_pred_dataset

In [6]:
# 3. Holiday avg results.
holiday_pred_sum = np.zeros((row_length, col_length))
holiday_pred_num = np.zeros((row_length, col_length))
for i in range(len(developing_pred_df)):
    holiday = int(developing_ft_df.iloc[i].at['holiday'])
    if holiday != 0:
        pred_obs = float(developing_pred_df.iat[i, 1])
        row = lat_max - developing_ft_df.iloc[i].at['latit']
        col = developing_ft_df.iloc[i].at['longit'] - lon_min
        holiday_pred_sum[row, col] = holiday_pred_sum[row, col] + pred_obs
        holiday_pred_num[row, col] = holiday_pred_num[row, col] + 1

holiday_pred_avg = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        if holiday_pred_num[row, col] > 0:
            holiday_pred_avg[row, col] = holiday_pred_sum[row, col]/float(holiday_pred_num[row, col])


# Save to tif
driver = gdal.GetDriverByName('GTiff')
holiday_pred_dataset = driver.Create(data_path + 'developing_holiday_avg_pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
holiday_pred_dataset.SetGeoTransform(tif_geotrans)
holiday_pred_dataset.SetProjection(tif_proj)
holiday_pred_dataset.GetRasterBand(1).WriteArray(holiday_pred_avg)
# Save finish.
del holiday_pred_dataset

In [7]:
# 4. More day example result - doy 126.
more_pred_sum = np.zeros((row_length, col_length))
more_pred_num = np.zeros((row_length, col_length))
for i in range(len(developing_pred_df)):
    doy = int(developing_ft_df.iloc[i].at['doy'])
    if doy == 126:
        pred_obs = float(developing_pred_df.iat[i, 1])
        row = lat_max - developing_ft_df.iloc[i].at['latit']
        col = developing_ft_df.iloc[i].at['longit'] - lon_min
        more_pred_sum[row, col] = more_pred_sum[row, col] + pred_obs
        more_pred_num[row, col] = more_pred_num[row, col] + 1

more_pred_avg = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        if more_pred_num[row, col] > 0:
            more_pred_avg[row, col] = more_pred_sum[row, col]/float(more_pred_num[row, col])


# Save to tif
driver = gdal.GetDriverByName('GTiff')
more_pred_dataset = driver.Create(data_path + 'developing_more_doy126_pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
more_pred_dataset.SetGeoTransform(tif_geotrans)
more_pred_dataset.SetProjection(tif_proj)
more_pred_dataset.GetRasterBand(1).WriteArray(more_pred_avg)
# Save finish.
del more_pred_dataset

In [ ]:
# 5. Less day example result - doy 176.
less_pred_sum = np.zeros((row_length, col_length))
less_pred_num = np.zeros((row_length, col_length))
for i in range(len(developing_pred_df)):
    doy = int(developing_ft_df.iloc[i].at['doy'])
    if doy == 176:
        pred_obs = float(developing_pred_df.iat[i, 1])
        row = lat_max - developing_ft_df.iloc[i].at['latit']
        col = developing_ft_df.iloc[i].at['longit'] - lon_min
        less_pred_sum[row, col] = less_pred_sum[row, col] + pred_obs
        less_pred_num[row, col] = less_pred_num[row, col] + 1

less_pred_avg = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        if less_pred_num[row, col] > 0:
            less_pred_avg[row, col] = less_pred_sum[row, col]/float(less_pred_num[row, col])

# Save to tif
driver = gdal.GetDriverByName('GTiff')
less_pred_dataset = driver.Create(data_path + 'developing_less_doy176_pred.tif', col_length, row_length, 1, gdal.GDT_Int32)
less_pred_dataset.SetGeoTransform(tif_geotrans)
less_pred_dataset.SetProjection(tif_proj)
less_pred_dataset.GetRasterBand(1).WriteArray(less_pred_avg)
# Save finish.
del less_pred_dataset

In [ ]:
# 6. Compare true observers and possible observers in developing region.
true_obs = gdal.Open(data_path + 'obs_blocks.tif')
true_obs_band = true_obs.GetRasterBand(1)
true_obs_data = true_obs.ReadAsArray()

possible_obs = gdal.Open(data_path + 'developing_2017pred.tif')
possible_obs_band = possible_obs.GetRasterBand(1)
possible_obs_data = possible_obs.ReadAsArray()

obs_gap = np.zeros((row_length, col_length))
for row in range(row_length):
    for col in range(col_length):
        if possible_obs_data[row, col] > 0:
            obs_gap[row, col] = possible_obs_data[row, col] - true_obs_data[row, col]

driver = gdal.GetDriverByName('GTiff')
gap_pred_dataset = driver.Create(data_path + 'developing_pred_increase.tif', col_length, row_length, 1, gdal.GDT_Int32)
gap_pred_dataset.SetGeoTransform(tif_geotrans)
gap_pred_dataset.SetProjection(tif_proj)
gap_pred_dataset.GetRasterBand(1).WriteArray(obs_gap)
# Save finish.
del gap_pred_dataset